# Tutorial 3: Three-Phase Systems with GAPoT

## Sistemas Trifásicos con GAPoT

This notebook demonstrates the application of Geometric Algebra to three-phase power systems,
including:

1. **Clarke transformation** (abc → αβ) using GA
2. **Park transformation** (αβ → dq) using rotors
3. **Symmetrical components** (Fortescue) in GA framework
4. **Unbalanced systems** analysis
5. **Power calculation** in three-phase systems

---

## Theoretical Background

### Three-Phase Representation

A balanced three-phase voltage system:
$$v_a(t) = V_m \sin(\omega t)$$
$$v_b(t) = V_m \sin(\omega t - 2\pi/3)$$
$$v_c(t) = V_m \sin(\omega t + 2\pi/3)$$

### Clarke Transform (abc → αβ)

The Clarke transform projects three-phase quantities onto a two-dimensional plane:

$$\begin{bmatrix} v_\alpha \\ v_\beta \end{bmatrix} = \frac{2}{3}\begin{bmatrix} 1 & -1/2 & -1/2 \\ 0 & \sqrt{3}/2 & -\sqrt{3}/2 \end{bmatrix} \begin{bmatrix} v_a \\ v_b \\ v_c \end{bmatrix}$$

**GA Interpretation**: This is a projection onto the plane orthogonal to the (1,1,1) direction.

### Park Transform (αβ → dq)

The Park transform rotates to a synchronously rotating reference frame:

$$\begin{bmatrix} v_d \\ v_q \end{bmatrix} = \begin{bmatrix} \cos\theta & \sin\theta \\ -\sin\theta & \cos\theta \end{bmatrix} \begin{bmatrix} v_\alpha \\ v_\beta \end{bmatrix}$$

**GA Interpretation**: This is rotation by rotor $R = e^{-B_{12}\theta/2}$ where $B_{12} = \sigma_\alpha \wedge \sigma_\beta$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Import GAPoT modules
from gapot import GeometricPower
from gapot.transforms import ClarkeTransform, ParkTransform, FortescueTransform, Rotor
from gapot.utils import generate_three_phase, rms

plt.style.use('seaborn-v0_8-whitegrid')
print("Modules loaded successfully!")

## 1. Generate Balanced Three-Phase System

In [ ]:
# System parameters
V_rms = 230.0      # Phase voltage RMS (V)
I_rms = 10.0       # Phase current RMS (A)
phi = np.pi/6      # Power factor angle (30° lagging)
f1 = 50.0          # Fundamental frequency (Hz)
fs = 10000.0       # Sampling frequency (Hz)
cycles = 2         # Number of cycles

# Generate signals
v_a, v_b, v_c, i_a, i_b, i_c, t = generate_three_phase(
    U=V_rms, I=I_rms, phi=phi, f1=f1, fs=fs, cycles=cycles
)

print(f"Phase voltage RMS: {rms(v_a):.2f} V")
print(f"Phase current RMS: {rms(i_a):.2f} A")
print(f"Power factor: {np.cos(phi):.3f}")

In [ ]:
# Plot three-phase voltages
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

t_ms = t * 1000

# Voltages
axes[0].plot(t_ms, v_a, 'r-', label='$v_a$', linewidth=1.5)
axes[0].plot(t_ms, v_b, 'g-', label='$v_b$', linewidth=1.5)
axes[0].plot(t_ms, v_c, 'b-', label='$v_c$', linewidth=1.5)
axes[0].set_ylabel('Voltage (V)')
axes[0].set_title('Three-Phase Voltages (Balanced System)')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Currents
axes[1].plot(t_ms, i_a, 'r-', label='$i_a$', linewidth=1.5)
axes[1].plot(t_ms, i_b, 'g-', label='$i_b$', linewidth=1.5)
axes[1].plot(t_ms, i_c, 'b-', label='$i_c$', linewidth=1.5)
axes[1].set_ylabel('Current (A)')
axes[1].set_xlabel('Time (ms)')
axes[1].set_title('Three-Phase Currents (30° Lagging)')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Clarke Transform (abc → αβ)

The Clarke transform projects the three-phase system onto a two-dimensional stationary reference frame.

In [ ]:
# Apply Clarke transform
clarke = ClarkeTransform(amplitude_invariant=False)

v_alpha, v_beta = clarke.forward(v_a, v_b, v_c)
i_alpha, i_beta = clarke.forward(i_a, i_b, i_c)

print(f"Voltage α RMS: {rms(v_alpha):.2f} V")
print(f"Voltage β RMS: {rms(v_beta):.2f} V")
print(f"Current α RMS: {rms(i_alpha):.2f} A")
print(f"Current β RMS: {rms(i_beta):.2f} A")

In [ ]:
# Plot αβ components
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Time domain
axes[0].plot(t_ms, v_alpha, 'b-', label='$v_\\alpha$', linewidth=1.5)
axes[0].plot(t_ms, v_beta, 'r-', label='$v_\\beta$', linewidth=1.5)
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Voltage (V)')
axes[0].set_title('Clarke Transform: αβ Components')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Space vector (αβ plane)
axes[1].plot(v_alpha, v_beta, 'b-', linewidth=0.5, alpha=0.7)
axes[1].scatter([v_alpha[0]], [v_beta[0]], c='g', s=100, marker='o', label='Start', zorder=5)
axes[1].set_xlabel('$v_\\alpha$ (V)')
axes[1].set_ylabel('$v_\\beta$ (V)')
axes[1].set_title('Space Vector Trajectory (αβ Plane)')
axes[1].set_aspect('equal')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nFor a balanced system, the space vector traces a perfect circle.")

## 3. Park Transform Using Rotors

### Traditional Matrix vs GA Rotor

**Matrix approach:**
$$\begin{bmatrix} v_d \\ v_q \end{bmatrix} = \begin{bmatrix} \cos\theta & \sin\theta \\ -\sin\theta & \cos\theta \end{bmatrix} \begin{bmatrix} v_\alpha \\ v_\beta \end{bmatrix}$$

**Rotor approach:**
$$R(t) = \cos(\theta/2) - B_{12}\sin(\theta/2) = e^{-B_{12}\theta/2}$$

The rotation is: $\mathbf{v}' = R\mathbf{v}\tilde{R}$

In [ ]:
# Apply Park transform
omega = 2 * np.pi * f1
park = ParkTransform(omega=omega)

v_d, v_q = park.forward(v_alpha, v_beta, t)
i_d, i_q = park.forward(i_alpha, i_beta, t)

print(f"Voltage d (mean): {np.mean(v_d):.2f} V")
print(f"Voltage q (mean): {np.mean(v_q):.2f} V")
print(f"Current d (mean): {np.mean(i_d):.2f} A")
print(f"Current q (mean): {np.mean(i_q):.2f} A")

In [ ]:
# Plot dq components
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# dq voltages
axes[0].plot(t_ms, v_d, 'b-', label='$v_d$', linewidth=1.5)
axes[0].plot(t_ms, v_q, 'r-', label='$v_q$', linewidth=1.5)
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Voltage (V)')
axes[0].set_title('Park Transform: dq Voltages (Should be DC)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# dq currents
axes[1].plot(t_ms, i_d, 'b-', label='$i_d$', linewidth=1.5)
axes[1].plot(t_ms, i_q, 'r-', label='$i_q$', linewidth=1.5)
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Current (A)')
axes[1].set_title('Park Transform: dq Currents (Should be DC)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nIn the synchronous reference frame, balanced sinusoidal quantities become DC.")
print(f"This confirms the rotor-based Park transform is working correctly.")

## 4. Rotor Visualization

Let's visualize how the rotor represents rotation geometrically.

In [ ]:
# Demonstrate rotor operation
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Create rotors at different angles
angles = np.linspace(0, 2*np.pi, 13)[:-1]  # 0° to 330° in 30° steps

# Starting vector
v_start = np.array([1.0, 0.0, 0.0])

# Plot rotations
colors = plt.cm.viridis(np.linspace(0, 1, len(angles)))

for idx, angle in enumerate(angles):
    rotor = Rotor(angle=angle, plane=(0, 1))
    v_rotated = rotor.apply(v_start)
    
    axes[0].arrow(0, 0, v_rotated[0]*0.9, v_rotated[1]*0.9, 
                  head_width=0.05, head_length=0.05, 
                  fc=colors[idx], ec=colors[idx], linewidth=2)
    
    # Label every 90°
    if idx % 3 == 0:
        axes[0].text(v_rotated[0]*1.1, v_rotated[1]*1.1, 
                     f'{int(np.degrees(angle))}°', fontsize=10)

axes[0].set_xlim(-1.5, 1.5)
axes[0].set_ylim(-1.5, 1.5)
axes[0].set_aspect('equal')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('Rotor Rotation: R·v·R̃')
axes[0].axhline(y=0, color='k', linewidth=0.5)
axes[0].axvline(x=0, color='k', linewidth=0.5)
axes[0].grid(True, alpha=0.3)

# Rotor components vs angle
angles_fine = np.linspace(0, 2*np.pi, 100)
scalar_parts = np.cos(angles_fine / 2)
bivector_parts = np.sin(angles_fine / 2)

axes[1].plot(np.degrees(angles_fine), scalar_parts, 'b-', 
             label='Scalar: $\\cos(\\theta/2)$', linewidth=2)
axes[1].plot(np.degrees(angles_fine), bivector_parts, 'r-', 
             label='Bivector: $\\sin(\\theta/2)$', linewidth=2)
axes[1].set_xlabel('Rotation Angle θ (degrees)')
axes[1].set_ylabel('Component Value')
axes[1].set_title('Rotor Components: R = cos(θ/2) - B·sin(θ/2)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(0, 360)

plt.tight_layout()
plt.show()

print("Key insight: The rotor uses HALF the rotation angle.")
print("This is why GA rotors provide double cover of SO(3) and avoid gimbal lock.")

## 5. Three-Phase Power Calculation

In [ ]:
# Calculate power for each phase using GAPoT
gp_a = GeometricPower(v_a, i_a, f1=f1, fs=fs)
gp_b = GeometricPower(v_b, i_b, f1=f1, fs=fs)
gp_c = GeometricPower(v_c, i_c, f1=f1, fs=fs)

# Total three-phase power
P_total = gp_a.P + gp_b.P + gp_c.P
Q_total = gp_a.M_Q_norm + gp_b.M_Q_norm + gp_c.M_Q_norm  # Simplified for balanced
S_total = np.sqrt(P_total**2 + Q_total**2)

print("THREE-PHASE POWER ANALYSIS")
print("=" * 50)
print(f"\n{'Phase':<10} {'P (W)':<12} {'Q (var)':<12} {'S (VA)':<12}")
print("-" * 50)
print(f"{'A':<10} {gp_a.P:<12.2f} {gp_a.M_Q_norm:<12.2f} {gp_a.S:<12.2f}")
print(f"{'B':<10} {gp_b.P:<12.2f} {gp_b.M_Q_norm:<12.2f} {gp_b.S:<12.2f}")
print(f"{'C':<10} {gp_c.P:<12.2f} {gp_c.M_Q_norm:<12.2f} {gp_c.S:<12.2f}")
print("-" * 50)
print(f"{'TOTAL':<10} {P_total:<12.2f} {Q_total:<12.2f} {S_total:<12.2f}")
print("=" * 50)

# Verify with traditional calculation
P_trad = 3 * V_rms * I_rms * np.cos(phi)
Q_trad = 3 * V_rms * I_rms * np.sin(phi)
print(f"\nTraditional calculation: P = 3VIcos(φ) = {P_trad:.2f} W")
print(f"Traditional calculation: Q = 3VIsin(φ) = {Q_trad:.2f} var")

## 6. Unbalanced System Analysis

In [ ]:
# Generate unbalanced system (10% voltage unbalance)
v_a_unb, v_b_unb, v_c_unb, i_a_unb, i_b_unb, i_c_unb, t_unb = generate_three_phase(
    U=V_rms, I=I_rms, phi=phi, f1=f1, fs=fs, cycles=cycles,
    unbalance=0.10  # 10% unbalance
)

print(f"Phase A voltage RMS: {rms(v_a_unb):.2f} V")
print(f"Phase B voltage RMS: {rms(v_b_unb):.2f} V")
print(f"Phase C voltage RMS: {rms(v_c_unb):.2f} V")

In [ ]:
# Clarke transform of unbalanced system
v_alpha_unb, v_beta_unb = clarke.forward(v_a_unb, v_b_unb, v_c_unb)

# Compare space vectors
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Balanced
axes[0].plot(v_alpha, v_beta, 'b-', linewidth=1, label='Balanced')
axes[0].set_xlabel('$v_\\alpha$ (V)')
axes[0].set_ylabel('$v_\\beta$ (V)')
axes[0].set_title('Balanced System: Circular Trajectory')
axes[0].set_aspect('equal')
axes[0].grid(True, alpha=0.3)

# Unbalanced
axes[1].plot(v_alpha_unb, v_beta_unb, 'r-', linewidth=1, label='Unbalanced (10%)')
axes[1].set_xlabel('$v_\\alpha$ (V)')
axes[1].set_ylabel('$v_\\beta$ (V)')
axes[1].set_title('Unbalanced System: Elliptical Trajectory')
axes[1].set_aspect('equal')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey observation: Unbalance causes the circular trajectory to become elliptical.")
print("This corresponds to the presence of negative sequence components.")

## 7. Symmetrical Components (Fortescue)

Fortescue decomposition separates unbalanced phasors into:
- **Positive sequence**: Balanced, rotating counterclockwise
- **Negative sequence**: Balanced, rotating clockwise  
- **Zero sequence**: In-phase components

In [ ]:
# Apply Fortescue transform
fortescue = FortescueTransform()

# Extract phasors (using FFT at fundamental)
from scipy.fft import fft, fftfreq

def extract_fundamental_phasor(signal, f1, fs):
    """Extract fundamental frequency phasor from signal."""
    N = len(signal)
    X = fft(signal)
    freqs = fftfreq(N, 1/fs)
    idx = np.argmin(np.abs(freqs - f1))
    return 2 * X[idx] / N

# Balanced system phasors
Va = extract_fundamental_phasor(v_a, f1, fs)
Vb = extract_fundamental_phasor(v_b, f1, fs)
Vc = extract_fundamental_phasor(v_c, f1, fs)

V_pos, V_neg, V_zero = fortescue.forward(Va, Vb, Vc)

print("BALANCED SYSTEM - Symmetrical Components")
print("=" * 50)
print(f"Positive sequence: |V+| = {np.abs(V_pos):.2f} V")
print(f"Negative sequence: |V-| = {np.abs(V_neg):.2f} V")
print(f"Zero sequence:     |V0| = {np.abs(V_zero):.2f} V")
print(f"\nVoltage Unbalance Factor (VUF): {100*np.abs(V_neg)/np.abs(V_pos):.2f}%")

# Unbalanced system phasors
Va_unb = extract_fundamental_phasor(v_a_unb, f1, fs)
Vb_unb = extract_fundamental_phasor(v_b_unb, f1, fs)
Vc_unb = extract_fundamental_phasor(v_c_unb, f1, fs)

V_pos_unb, V_neg_unb, V_zero_unb = fortescue.forward(Va_unb, Vb_unb, Vc_unb)

print("\n" + "=" * 50)
print("UNBALANCED SYSTEM (10%) - Symmetrical Components")
print("=" * 50)
print(f"Positive sequence: |V+| = {np.abs(V_pos_unb):.2f} V")
print(f"Negative sequence: |V-| = {np.abs(V_neg_unb):.2f} V")
print(f"Zero sequence:     |V0| = {np.abs(V_zero_unb):.2f} V")
print(f"\nVoltage Unbalance Factor (VUF): {100*np.abs(V_neg_unb)/np.abs(V_pos_unb):.2f}%")

## 8. Key Takeaways

1. **Clarke transform** projects three-phase to αβ plane (stationary reference frame)
2. **Park transform** rotates to dq frame using **rotors** instead of matrices
3. **Rotor advantages**: Explicit rotation plane, simple inverse, natural interpolation
4. **Space vector trajectory**: Circular for balanced, elliptical for unbalanced
5. **Symmetrical components** decompose unbalance into positive/negative/zero sequences
6. **GA provides unified framework** for all these transformations